## Strategy Idea 2 : "Comovement - pairs trading"

__Section 0: Setup__ Importing packages/reading in data etc.

__Section 1 : Idea__ 

- __1.1__ Strategy idea

- __1.2__ Origin of idea. Context/Reasoning for strategy to work e.g. use in financial markets?

__Section 2 : Exploration__

- __2.1__ Exploratory Data Analysis. e.g plots of price/volumes that could show strategy working, how much potential.

- __2.2__ Define some 'strategy metrics'. Metrics that can can you use to gauge if this strategy will work i.e no.price points above a certain threshold that is profitable. Metrics could show how often there is an opportunity to make a trade and how much 'value' is in an opportunity e.g. how much is there a price swing?


__Section 3 : Strategy testing__

- __3.1__ Testing strategy on previous data. 

- __3.2__ State any assumptions made by testing.

- __3.3__ Model refinements. How could strategy be optimised? Careful : is this backfitting/overfitting - what measures taken to negate this e.g. bootstrapping?

- __3.4__ Assessing strategy. P/L on data sample? ROI? variance in results? longest losing run?

__Section 4 : Practical requirements__

- __4.1__ Identify if this edge is ‘realisable’? What methods will you apply to extract this value? e.g. applying a hedge function


- __4.2__ Is it possible to quantify the potential profit from the strategy? Consideration : How long will it take to obtain this? How 'risky' is it? e.g. if something did go wrong, how much do we lose? 

- __4.3__ Strategy limitations. The factors that could prevent strategy working e.g. practical considerations e.g. reacting quick enough to market updates, volume behind a price, size of bankroll needed


__Section 5: Potential limitations__

- __5.1__ What is our 'competition' - if not quantifiable, do we suspect people are doing the same thing? 

- __5.2__ So what's our edge? Identify ways of finding this edge in future? e.g what features are there? Are they predictive? Is there a certain 'market/runner' profile?





### Section 0 : Setup

In [5]:
# importing packages
from pathlib import Path, PurePath 

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import utils

In [6]:
# reading in data
project_dir = Path.cwd().parents[2]
data_dir = project_dir / 'data' / 'processed' / 'api' / 'advanced' / 'adv_data.csv'
df = pd.read_csv(data_dir, index_col = 0)
print(df.shape)
df.head()

(13073, 307)


,SelectionId,MarketId,Venue,Distance,RaceType,BSP,NoRunners,BS:T-60,BS:T-59,BS:T-58,...,LS:T+5,LS:T+6,LS:T+7,LS:T+8,LS:T+9,LS:T+10,LS:T+11,LS:T+12,LS:T+13,LS:T+14
0,11986132,1.169028,Huntingdon,20.0,Chase,8.33,9,16.43,24.51,26.57,...,10.08,11.15,5.44,7.09,14.16,19.53,3.12,3.31,0.68,0.68
1,16800725,1.169028,Huntingdon,20.0,Chase,3.68,9,15.43,25.74,57.82,...,29.87,221.22,43.23,43.10,13.53,26.15,13.60,74.30,419.52,23082.10
2,20968322,1.169028,Huntingdon,20.0,Chase,14.96,9,9.87,9.25,9.15,...,37.32,6.83,4.85,11.23,16.00,5.68,40.25,12.51,10.42,13.17
3,22023486,1.169028,Huntingdon,20.0,Chase,4.25,9,84.38,64.49,58.01,...,11.67,2.02,2.02,2.02,2.02,2.02,2.02,2.02,2.02,2.02
4,24496216,1.169028,Huntingdon,20.0,Chase,6.60,9,10.64,10.11,7.91,...,34.27,54.72,11.85,17.99,48.21,17.28,38.29,6.96,4.37,4.37


### Section 1 : Idea

__1.1 Idea__

Prices in a market adjust such that the sum of the implied odds of all horses is approximately equal to one. If one horse's price drifts, another's or several other's should be backed in. 

One traditional strategy considering this price behaviour is 'pairs trading': *"A pairs trade or pair trading is a market neutral trading strategy enabling traders to profit from virtually any market conditions: uptrend, downtrend, or sideways movement. This strategy is categorized as a statistical arbitrage and convergence trading strategy."*

__1.2  Reasoning__

Why is there an edge here?
- .

### Section 2 : Exploration

__2.1__ Sample covariance

To explore this phenomena, an example covariance matrix from a random sample race is taken below.

In [7]:
#transpose sample df
sample_df = df[df['MarketId'] == df['MarketId'].sample(1).item()] 

transpose_df = sample_df.T.copy()
transpose_df.columns = ["h" + str(column) for column in transpose_df.iloc[0]]
transpose_df = transpose_df.iloc[7:]
transpose_df.reset_index(drop=True, inplace=True)
transpose_df = transpose_df.astype(float)

transpose_df.head()

,h11244806,h12090523,h12192145,h12845992,h13306150,h13957361,h18705419,h19450775,h5542918,h8487860,h8971553
0,39.14,9.06,72.66,8.46,24.79,4.81,7.27,8.82,8.32,10.64,8.63
1,18.99,9.56,56.09,8.85,18.22,5.70,5.79,8.33,9.90,2.91,9.77
2,12.62,3.23,71.70,9.74,15.48,4.26,7.71,10.25,10.25,4.21,6.77
3,18.92,6.27,32.39,10.08,12.68,4.06,8.20,11.94,11.63,23.68,8.74
4,19.32,7.59,39.45,9.74,12.16,1.40,5.71,13.23,12.65,22.08,8.58


In [8]:
transpose_df.cov()

,h11244806,h12090523,h12192145,h12845992,h13306150,h13957361,h18705419,h19450775,h5542918,h8487860,h8971553
h11244806,5057.596068,3042.614034,5220.206670,3978.791071,2982.875886,2305.755290,4389.948645,3687.103195,1968.640336,3787.984341,2457.141164
h12090523,3042.614034,18317.612660,-4326.686980,5618.978720,13682.432090,11435.612581,6782.301107,8997.058380,16847.349969,6883.382656,17789.712905
h12192145,5220.206670,-4326.686980,739633.152741,-2208.945746,-2202.694772,-11429.279037,119.705006,-4098.499676,-8684.285836,-2009.190510,-7529.805195
h12845992,3978.791071,5618.978720,-2208.945746,6235.580012,4539.205400,6826.755501,6305.753799,6572.609879,4745.586164,5965.035214,5660.239183
h13306150,2982.875886,13682.432090,-2202.694772,4539.205400,11952.527178,8948.579922,5554.804698,7239.828497,12531.551305,5547.805481,13455.474538
h13957361,2305.755290,11435.612581,-11429.279037,6826.755501,8948.579922,18090.650796,7995.929330,11692.748348,10867.699546,8774.623510,13134.742557
h18705419,4389.948645,6782.301107,119.705006,6305.753799,5554.804698,7995.929330,7338.642290,7965.358820,5352.951808,7002.657954,6553.675429
h19450775,3687.103195,8997.058380,-4098.499676,6572.609879,7239.828497,11692.748348,7965.358820,10151.350927,8028.595610,8203.302551,9221.890312
h5542918,1968.640336,16847.349969,-8684.285836,4745.586164,12531.551305,10867.699546,5352.951808,8028.595610,27540.778324,5626.284885,16801.267716
h8487860,3787.984341,6883.382656,-2009.190510,5965.035214,5547.805481,8774.623510,7002.657954,8203.302551,5626.284885,7200.873796,6903.607076


In [9]:
transpose_df.corr()

,h11244806,h12090523,h12192145,h12845992,h13306150,h13957361,h18705419,h19450775,h5542918,h8487860,h8971553
h11244806,1.000000,0.316112,0.085351,0.708501,0.383648,0.241054,0.720576,0.514578,0.166804,0.627688,0.255388
h12090523,0.316112,1.000000,-0.037172,0.525756,0.924696,0.628200,0.584972,0.659788,0.750083,0.599342,0.971575
h12192145,0.085351,-0.037172,1.000000,-0.032527,-0.023427,-0.098806,0.001625,-0.047299,-0.060847,-0.027531,-0.064717
h12845992,0.708501,0.525756,-0.032527,1.000000,0.525789,0.642760,0.932161,0.826109,0.362129,0.890189,0.529832
h13306150,0.383648,0.924696,-0.023427,0.525789,1.000000,0.608551,0.593104,0.657259,0.690696,0.597997,0.909727
h13957361,0.241054,0.628200,-0.098806,0.642760,0.608551,1.000000,0.693959,0.862835,0.486881,0.768792,0.721833
h18705419,0.720576,0.584972,0.001625,0.932161,0.593104,0.693959,1.000000,0.922859,0.376528,0.963302,0.565482
h19450775,0.514578,0.659788,-0.047299,0.826109,0.657259,0.862835,0.922859,1.000000,0.480164,0.959476,0.676550
h5542918,0.166804,0.750083,-0.060847,0.362129,0.690696,0.486881,0.376528,0.480164,1.000000,0.399522,0.748335
h8487860,0.627688,0.599342,-0.027531,0.890189,0.597997,0.768792,0.963302,0.959476,0.399522,1.000000,0.601347


### Section 3 : Pairs trading

__3.1__ **- Idea**

[Bebbington, PA (2017)](https://discovery.ucl.ac.uk/id/eprint/1563501/) looks at pairs trading in horse racing markets. The following outlines their method for analysing this strategy. In 3.2, each step will be attempted.

* The 'signals' are the best match or lay price available at a given timestamp.
* Statsitical methods are used to analyse horses' pricing data for comparison, in particular to calcuate a hedge ratio and for stake weighting. In the paper, non-overlapping windows of data, for example, price observations 1-5, 6-10, 11-15, make up the time series, and then trades are made at the end of the window. This is used to simulate a method where the algorithm is reacting to live data. This example study movement throughout 30 price points and make bets in the remaining 30 periods.
* A z-score transformation of the log of the decimal odds is used to standardise prices. This makes the relative directional movement in different prices comparable by accounting for their respective variances. 
* Pairs are discovered by analysing the sum of squared distances between two horses' prices throughout time. Those that move the least relative to each other are the best candidates for pairs.
* Once pairs are identified, the 'spread' between their prices (on average, or at the end of each window) is compared to a minimum size requirement for a bet to be made, $\phi$.
* The 'hedging ratio' is found using an OLS regression of the price of one of the horses on the price of the other. Since the two prices are pairs but will have different variances and absolute values, their movement relative to eachother must be considered to make the strategy 'cost neutral'. It is also used to define the stake size. 
* The final observed spread indicates which on which horse a 'back-to-lay' hedge must be made (that which is expected to be backed in) and on which a 'lay-to-back' hedge must be made (that which is expected to drift). This spread is compared to an interval [?], likely a confidence interval of past spreads or simply the interval of observed spreads. If the spread is greater than usual or smaller than usual, the bets are placed. 
* In the paper it appears that both sides of the hedge bet are made at the same point in time.

__3.2__ **- Setup**

**Data**

The following example will be set up with a random race and will identify tradeable pairs (or that there are none). Three DataFrames are created: (1) the unchanged race sample DataFrame with one row per horse and data going along in columns, (2) a back prices DataFrame with one column per horse and prices going through time in rows, (3) the same for lay prices. This analysis looks at prices before the race begins.

There are 60 price data points for each horse, finishing at the begining of the race.

Variables:
* $BP_{t}^{i}$ is back price for horse i at time t.
* $LP_{t}^{i}$ is lay price for horse i at time t.

In [116]:
# defining variables
back_prices = [col for col in df.columns if 'BP' in col]
back_sizes = [col for col in df.columns if 'BS' in col]
lay_prices = [col for col in df.columns if 'LP' in col]
lay_sizes = [col for col in df.columns if 'LS' in col]

#runner_info = ['SelectionId', 'MarketId', 'Venue', 'Distance', 'RaceType', 'BSP', 'NoRunners']

sample_df = df[df['MarketId'] == df['MarketId'].sample(1).item()] 

bp_df = sample_df[['SelectionId'] + back_prices].copy()
new_cols = bp_df.columns.str.replace("[BP:T]", "").str.replace("[+]", "")
bp_df.rename(columns = dict(zip(bp_df.columns, new_cols)), inplace = True)
bp_t_df = bp_df.T.copy()
bp_t_df.columns = ["h" + str(column) for column in bp_t_df.iloc[0]]
bp_t_df = bp_t_df.iloc[1:-15] # using the 60 pre-off price data points
bp_t_df.reset_index(drop=True, inplace=True)

lp_df = sample_df[['SelectionId'] + lay_prices].copy()
new_cols = lp_df.columns.str.replace("[LP:T]", "").str.replace("[+]", "")
lp_df.rename(columns = dict(zip(lp_df.columns, new_cols)), inplace = True)
lp_t_df = lp_df.T.copy()
lp_t_df.columns = ["h" + str(column) for column in lp_t_df.iloc[0]]
lp_t_df = lp_t_df.iloc[1:-15]
lp_t_df.reset_index(drop=True, inplace=True)

# bsp_df = plot_df[['BSP']].copy()
# bsp_df['min_bp'] = bsp_df['BSP'].apply(lambda x: round(utils.back_hedge_min_bp(x, 0.05), 2))
# bsp_df['max_lp'] = bsp_df['BSP'].apply(lambda x: round(utils.lay_hedge_max_lp(x, 0.05), 2))    

bp_t_df.head()

,h12905345.0,h12925614.0,h18662255.0,h18976983.0,h23510430.0,h3324323.0,h665641.0
0,4.55,7.49,7.40,10.0,3.51,18.50,11.49
1,4.50,7.52,7.40,10.0,3.48,18.58,11.50
2,4.53,7.42,7.40,10.0,3.50,17.84,11.58
3,4.56,7.70,7.46,10.0,3.45,18.58,11.53
4,4.60,7.72,7.49,10.0,3.37,19.16,11.22


__3.3__ **- Z-score transformation**

Bebbington standardises prices by taking the natural logarithm of each price and then standardising it with a Z-score transformation. 

Taking $P_{t}^{i} = BP_{t}^{i}$, this means finding 

#### $P_{t}^{'(i)} = \frac{P_{t}^{i}-\overline{P}_{t}^{i}}{\sigma^{(i)}}$

where $\sigma^{(i)}$ is the standard deviation of the horse's price throughout the time series.

The Z-score transformation gives the relationship between an individual data point in the sample relative to that of the population mean and standard deviation. This means that variations are comparable between horses.



In [117]:
z_bp_df = bp_t_df.copy()
z_bp_df = np.log(z_bp_df)

for column in z_bp_df.columns:
    mean = z_bp_df[column].mean()
    sd = np.std(z_bp_df[column], ddof = 1)
    z_bp_df[column] = z_bp_df[column].apply(lambda x: (x - mean) / sd)
    
z_bp_df.head()

,h12905345.0,h12925614.0,h18662255.0,h18976983.0,h23510430.0,h3324323.0,h665641.0
0,1.265380,-0.686342,-0.919074,-0.066906,0.513760,-0.943338,1.397800
1,0.958437,-0.592561,-0.919074,-0.066906,0.225867,-0.904000,1.420266
2,1.143009,-0.906634,-0.919074,-0.066906,0.418070,-1.274515,1.599293
3,1.326363,-0.037613,-0.822694,-0.066906,-0.064519,-0.904000,1.487547
4,1.568968,0.023245,-0.774794,-0.066906,-0.851401,-0.623771,0.783711


__3.4__ **- Sum of squared distances**

Following Bebbington, to select pairs, we create a matrix (DataFrame, in this case) of the sum of squared distances between pairs of horses throughout the time series.

$\Theta _{ij} = \left\{\begin{matrix}
\sum_{M}^{t=1}(P_{t}^{'(i)} - P_{t}^{'(j)})^{2}, &i\neq j\\ 
 0, i=j& 
\end{matrix}\right.$

In [118]:
ids = [column for column in z_bp_df.columns]

matrix = z_bp_df.iloc[0:0].copy()

matrix.insert(0, "horse", np.array(ids))
matrix = matrix.set_index("horse", drop = False)
del matrix["horse"]

for column in matrix.columns:
    for row in matrix.index:
        if column == row:
            matrix.loc[row, column] = np.nan
        else: 
            matrix.loc[row, column] = ((z_bp_df[row] - z_bp_df[column]) ** 2).sum() or np.nan
            
for x in range(len(ids)):
    for y in range(x, len(ids)):
        matrix.iloc[x, y] = np.nan
        
matrix

,h12905345.0,h12925614.0,h18662255.0,h18976983.0,h23510430.0,h3324323.0,h665641.0
horse,,,,,,,
h12905345.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h12925614.0,192.018237,NaN,NaN,NaN,NaN,NaN,NaN
h18662255.0,127.744531,187.795395,NaN,NaN,NaN,NaN,NaN
h18976983.0,91.021036,152.850338,129.788179,NaN,NaN,NaN,NaN
h23510430.0,150.331527,34.092123,208.232771,150.710881,NaN,NaN,NaN
h3324323.0,186.492394,75.511588,109.351975,135.909367,125.491142,NaN,NaN
h665641.0,109.254593,120.483611,137.292934,130.036075,99.832559,189.157642,NaN


In [119]:
horse_x = matrix.min(axis=1).idxmin()
horse_y = matrix.min().idxmin()
sss = matrix.min().min()

print(f"Pair found: horse {horse_x} and horse {horse_y} with sum of squared spreads equal to {sss}.")

Pair found: horse h23510430.0 and horse h12925614.0 with sum of squared spreads equal to 34.09212258936087.


__3.4__ **- Regression of prices of horse Y on horse X**

With $x_{t} = \left \{P_{1}^{X} + P_{2}^{X} + , ... , P_{M}^{X} \right \}$ and  $y_{t} = \left \{P_{1}^{Y} + P_{2}^{Y} + , ... , P_{M}^{Y} \right \}$ where $M$ is the final time period in the window, we carry out the OLS regression of $y_{t}$ on $x_{t}$. The estimate of $\beta$ is the hedging ratio, giving the relative holding of $x_{t}$ for a cost-neutral hedge position.

$y_{t} = \beta x_{t} + \varepsilon_{t}$

Using this estimation and the final end of window observations at time $M$ we get the spread at the end of the window.

$\varepsilon_{M} = y_{M} - \hat{\beta} x_{M}$.

If $\varepsilon_{M}$ is outside of an interval of past spread values such that if the spread returns to the mean a hedge bet will be profitable, bets can be made.